<a href="https://colab.research.google.com/github/adenurchalisa/Automatic-Photo-Clustering-System-Optimization-HDBSCAN/blob/main/notebooks/12_CGA_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 12: CGA — Cluster-Guided Gaussian Augmentation

## Latar Belakang

Berdasarkan analisis di **Notebook 11**, ditemukan bahwa:
- Baseline (Correlation, mcs=15, ms=70) menghasilkan **54 cluster** dengan noise **43.8%**
- Dari total noise, **25% adalah probable singletons** (CGA candidates)
- Ada **minority clusters** (ukuran kecil) yang rentan hilang saat HDBSCAN dijalankan dengan threshold berbeda

## Apa itu CGA?

**CGA (Cluster-Guided Gaussian Augmentation)** adalah adaptasi dari prinsip inti FICAug (Haghbin et al., 2025) untuk konteks *unsupervised* face clustering:

> Untuk setiap minority cluster di embedding space, buat sampel sintetis baru menggunakan **Gaussian sampling** di sekitar distribusi cluster tersebut — sehingga cluster yang terlalu kecil untuk di-detect HDBSCAN menjadi cukup "padat" untuk terbentuk.

### Formula:
$$\mathbf{x}_{syn} = \boldsymbol{\mu}_k + \alpha \cdot \boldsymbol{\sigma}_k \cdot \epsilon, \quad \epsilon \sim \mathcal{N}(0, I)$$

Di mana:
- $\boldsymbol{\mu}_k$ = centroid cluster ke-$k$ (GPU: `cp.mean`)
- $\boldsymbol{\sigma}_k$ = std per dimensi cluster ke-$k$ (diagonal approximation, GPU: `cp.std`)
- $\alpha$ = scale factor yang mengontrol variasi (hyperparameter)
- $\epsilon$ = random noise dari distribusi normal standar (GPU: `cp.random.randn`)

## Alur Eksperimen

```
Embeddings (12,715 × 512)
        ↓
Baseline Clustering (Correlation, mcs=15, ms=70)
        ↓
Identifikasi Minority Clusters
        ↓
CGA: Gaussian Sampling per Cluster [GPU - CuPy]
        ↓
Gabungkan: Original + Synthetic Embeddings
        ↓
Re-clustering HDBSCAN
        ↓
Evaluasi & Ablation Study (α, n_synthetic)
```

## Akselerasi GPU

Seluruh operasi statistik (mean, std, random sampling) dilakukan di GPU menggunakan **CuPy** — konsisten dengan pendekatan di Notebook 6 (QJSD) dan Notebook 10 (Angular Distance).

---

## Cell 1 — Instalasi & Imports

In [ ]:
!pip install hdbscan cupy-cuda12x -q

import numpy as np
import pickle
import time
import os
import warnings
from collections import Counter
from datetime import datetime
from itertools import product

import cupy as cp
import hdbscan
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator

from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

# Cek GPU
gpu_name = cp.cuda.runtime.getDeviceProperties(0)['name'].decode()
gpu_mem  = cp.cuda.runtime.getDeviceProperties(0)['totalGlobalMem'] / (1024**3)
print(f"✅ GPU: {gpu_name}  ({gpu_mem:.1f} GB VRAM)")
print(f"✅ CuPy version : {cp.__version__}")
print(f"✅ HDBSCAN version: {hdbscan.__version__}")
print("✅ Semua library berhasil dimuat")

## Cell 2 — Konfigurasi

In [ ]:
class Config:
    # ─── Path ────────────────────────────────────────────────────────
    EMBEDDINGS_PATH  = '/content/drive/MyDrive/OTW S.KOM/Embeddings/embeddings_data.pkl'
    NB11_RESULTS     = '/content/drive/MyDrive/OTW S.KOM/Results/notebook11_noise_analysis.pkl'
    RESULTS_DIR      = '/content/drive/MyDrive/OTW S.KOM/Results/'
    PLOTS_DIR        = '/content/drive/MyDrive/OTW S.KOM/Results/Plots/'

    # ─── Baseline HDBSCAN (sama dengan Notebook 11) ──────────────────
    BASELINE_PARAMS = {
        'metric'           : 'correlation',
        'min_cluster_size' : 15,
        'min_samples'      : 70,
        'method'           : 'eom',
    }

    # ─── Minority Cluster Threshold ──────────────────────────────────
    # Cluster dengan ukuran <= threshold ini dianggap 'minority'
    # dan menjadi target CGA augmentation.
    # Default: 2 × min_cluster_size (30)
    MINORITY_THRESHOLD = 30

    # ─── CGA Hyperparameters ─────────────────────────────────────────
    # Nilai alpha yang akan di-ablasi
    ALPHA_LIST      = [0.5, 1.0, 1.5]

    # Jumlah sampel sintetis per cluster member yang di-ablasi
    N_SYNTHETIC_LIST = [3, 5, 10]

    # Seed reproducibility di GPU
    SEED = 42

    # ─── Re-clustering params ────────────────────────────────────────
    # Gunakan parameter yang sama dengan baseline untuk fair comparison
    RECLUSTER_PARAMS = {
        'metric'           : 'correlation',
        'min_cluster_size' : 15,
        'min_samples'      : 70,
        'method'           : 'eom',
    }

    # Batch size untuk transfer GPU → CPU
    GPU_BATCH_SIZE = 2000


cp.random.seed(Config.SEED)

print("=" * 60)
print("✅ CONFIG LOADED")
print("=" * 60)
print(f"\n  Minority threshold  : ≤ {Config.MINORITY_THRESHOLD} members")
print(f"  Alpha values (ablasi): {Config.ALPHA_LIST}")
print(f"  N synthetic (ablasi) : {Config.N_SYNTHETIC_LIST}")
print(f"  GPU Seed             : {Config.SEED}")
print(f"  Total kombinasi ablasi: {len(Config.ALPHA_LIST) * len(Config.N_SYNTHETIC_LIST)}")

## Cell 3 — Load Data & Reproduce Baseline

Data embedding dimuat dan HDBSCAN baseline dijalankan ulang — identik dengan konfigurasi di Notebook 11 (Correlation, mcs=15, ms=70). Ini memastikan konsistensi eksperimen.

In [ ]:
from google.colab import drive

print("=" * 60)
print("📁 LOAD DATA")
print("=" * 60)

if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive')
else:
    print("✅ Google Drive sudah ter-mount")

os.makedirs(Config.RESULTS_DIR, exist_ok=True)
os.makedirs(Config.PLOTS_DIR, exist_ok=True)

# Load embeddings
with open(Config.EMBEDDINGS_PATH, 'rb') as f:
    data = pickle.load(f)

embeddings = np.array(data['embeddings']).astype('float32')
metadata   = data['metadata']

print(f"\n✅ Embeddings loaded: {embeddings.shape}")
print(f"   Metadata: {len(metadata)} entries")

# Transfer ke GPU
print(f"\n⏳ Transfer embeddings ke GPU...")
t = time.time()
embeddings_gpu = cp.array(embeddings)  # (N, 512) di VRAM
print(f"   ✅ GPU array: {embeddings_gpu.shape}, dtype={embeddings_gpu.dtype} ({time.time()-t:.2f}s)")
vram_mb = embeddings_gpu.nbytes / (1024**2)
print(f"   VRAM digunakan: {vram_mb:.1f} MB")

# Reproduce baseline clustering (CPU - HDBSCAN belum support correlation di cuML)
print(f"\n⏳ Reproducing baseline (Correlation, mcs=15, ms=70)...")
t = time.time()
clusterer_baseline = hdbscan.HDBSCAN(
    min_cluster_size         = Config.BASELINE_PARAMS['min_cluster_size'],
    min_samples              = Config.BASELINE_PARAMS['min_samples'],
    cluster_selection_method = Config.BASELINE_PARAMS['method'],
    metric                   = Config.BASELINE_PARAMS['metric'],
)
labels_baseline = clusterer_baseline.fit_predict(embeddings)
elapsed = time.time() - t

n_clusters_base = len(set(labels_baseline)) - 1
n_noise_base    = int((labels_baseline == -1).sum())
coverage_base   = float((labels_baseline != -1).sum() / len(labels_baseline))

print(f"   ✅ Selesai ({elapsed:.2f}s)")
print(f"\n  ┌──────────────────────────────────────────┐")
print(f"  │  BASELINE RESULTS                        │")
print(f"  ├──────────────────────────────────────────┤")
print(f"  │  Clusters     : {n_clusters_base:<25}│")
print(f"  │  Noise        : {n_noise_base:<8,} ({1-coverage_base:.1%})            │")
print(f"  │  Coverage     : {coverage_base:.1%}                       │")
print(f"  └──────────────────────────────────────────┘")

## Cell 4 — Identifikasi Minority Clusters

Minority clusters adalah cluster dengan jumlah anggota di bawah **`MINORITY_THRESHOLD`** (default: 30, atau 2× min_cluster_size). Cluster-cluster ini menjadi target augmentasi CGA.

Logika: jika sebuah cluster punya terlalu sedikit anggota, HDBSCAN menjadi tidak stabil dalam mendeteksinya — sedikit perubahan hyperparameter bisa membuatnya hilang. CGA menstabilkan cluster ini dengan menambah density di sekitarnya.

In [ ]:
print("=" * 60)
print("🔍 IDENTIFIKASI MINORITY CLUSTERS")
print("=" * 60)

# Hitung ukuran setiap cluster
cluster_sizes = Counter({k: v for k, v in Counter(labels_baseline).items() if k != -1})
sizes_arr     = np.array(list(cluster_sizes.values()))
cluster_ids   = np.array(list(cluster_sizes.keys()))

# Identifikasi minority clusters
minority_ids   = cluster_ids[sizes_arr <= Config.MINORITY_THRESHOLD]
majority_ids   = cluster_ids[sizes_arr > Config.MINORITY_THRESHOLD]

n_minority = len(minority_ids)
n_majority = len(majority_ids)

minority_members_total = sum(cluster_sizes[k] for k in minority_ids)
majority_members_total = sum(cluster_sizes[k] for k in majority_ids)

print(f"""
  Threshold minority : ≤ {Config.MINORITY_THRESHOLD} anggota

  ┌─────────────────────────────────────────────────────────────┐
  │  DISTRIBUSI CLUSTER                                         │
  ├─────────────────────────────────────────────────────────────┤
  │  Total clusters      : {n_clusters_base:<6}                          │
  │  Majority clusters   : {n_majority:<6} ({n_majority/n_clusters_base:.1%}) — TIDAK di-augment  │
  │  Minority clusters   : {n_minority:<6} ({n_minority/n_clusters_base:.1%}) — Target CGA        │
  ├─────────────────────────────────────────────────────────────┤
  │  Anggota di majority : {majority_members_total:<6,}                          │
  │  Anggota di minority : {minority_members_total:<6,}                          │
  └─────────────────────────────────────────────────────────────┘
""")

print("  Detail minority clusters:")
print(f"  {'Cluster ID':>10} {'Ukuran':>8}")
print(f"  {'─'*10} {'─'*8}")
for k in sorted(minority_ids):
    print(f"  {k:>10} {cluster_sizes[k]:>8}")

print(f"\n  Statistik ukuran cluster (seluruh):")
print(f"  Min : {sizes_arr.min()}")
print(f"  Max : {sizes_arr.max()}")
print(f"  Mean: {sizes_arr.mean():.1f}")
print(f"  Std : {sizes_arr.std():.1f}")

## Cell 5 — CGA Core: Gaussian Sampling di GPU

Fungsi utama CGA yang sepenuhnya berjalan di GPU menggunakan CuPy.

### Algoritma:
```
Untuk setiap minority cluster k:
    1. Ambil semua embedding anggota cluster k → cluster_embs (CuPy array)
    2. Hitung µ_k = mean(cluster_embs)  [GPU]
    3. Hitung σ_k = std(cluster_embs)   [GPU, diagonal approx]
    4. Hitung r_k = max distance dari µ_k (radius cluster)  [GPU]
    5. Untuk i in range(n_synthetic):
        a. Sample ε ~ N(0, I)  [GPU]
        b. x_syn = µ_k + α * σ_k * ε  [GPU]
        c. Validasi: ||x_syn - µ_k|| <= r_k  [GPU]
        d. Jika valid → tambahkan ke synthetic pool
```

### Kenapa diagonal approximation untuk σ?
Full covariance matrix untuk embedding 512-dim berukuran 512×512 = 262,144 parameter per cluster. Dengan diagonal approximation, kita hanya butuh 512 nilai — jauh lebih efisien dan cukup akurat untuk distribusi embeddings wajah.

In [ ]:
def cga_gaussian_sampling_gpu(
    embeddings_gpu: cp.ndarray,
    labels: np.ndarray,
    target_cluster_ids: np.ndarray,
    alpha: float = 1.0,
    n_synthetic: int = 5,
    seed: int = 42
) -> tuple[cp.ndarray, np.ndarray]:
    """
    CGA: Cluster-Guided Gaussian Augmentation — sepenuhnya berjalan di GPU.

    Parameters
    ----------
    embeddings_gpu     : CuPy array (N, D) — semua embedding di VRAM
    labels             : NumPy array (N,) — label cluster HDBSCAN
    target_cluster_ids : cluster ID yang akan di-augment
    alpha              : scale factor variasi (0.5=kecil, 1.0=normal, 1.5=besar)
    n_synthetic        : jumlah sampel sintetis yang dihasilkan per cluster
    seed               : random seed untuk reproducibility

    Returns
    -------
    syn_embs_gpu   : CuPy array (M, D) — synthetic embeddings di VRAM
    syn_labels     : NumPy array (M,) — label cluster dari synthetic embeddings

    Notes
    -----
    - Diagonal covariance approximation: σ_k = std per dimensi (bukan full cov matrix)
    - Boundary validation: hanya sampel dalam radius cluster r_k yang disimpan
    - Seluruh operasi (mean, std, randn, distance) berjalan di GPU
    """
    cp.random.seed(seed)

    syn_embs_list   = []
    syn_labels_list = []

    labels_gpu = cp.array(labels)  # transfer label ke GPU untuk masking
    D = embeddings_gpu.shape[1]

    for k in target_cluster_ids:
        # --- Step 1: Ambil anggota cluster k di GPU ---
        mask         = labels_gpu == int(k)
        cluster_embs = embeddings_gpu[mask]          # (n_k, D)
        n_k          = cluster_embs.shape[0]

        if n_k < 2:
            # Tidak cukup untuk estimasi distribusi
            continue

        # --- Step 2 & 3: Estimasi distribusi Gaussian [GPU] ---
        mu    = cluster_embs.mean(axis=0)            # (D,) — centroid
        sigma = cluster_embs.std(axis=0)             # (D,) — diagonal std

        # Hindari sigma = 0 (dimensi konstan dalam cluster)
        sigma = cp.where(sigma < 1e-8, 1e-8, sigma)

        # --- Step 4: Hitung radius cluster r_k [GPU] ---
        diffs      = cluster_embs - mu[cp.newaxis, :]  # (n_k, D)
        distances  = cp.linalg.norm(diffs, axis=1)     # (n_k,)
        r_k        = float(distances.max())             # scalar

        # --- Step 5: Sample n_synthetic sampel sekaligus [GPU] ---
        epsilon = cp.random.randn(n_synthetic, D)                  # (n_syn, D)
        x_syn   = mu[cp.newaxis, :] + alpha * sigma[cp.newaxis, :] * epsilon  # (n_syn, D)

        # --- Boundary validation [GPU] ---
        syn_dists  = cp.linalg.norm(x_syn - mu[cp.newaxis, :], axis=1)  # (n_syn,)
        valid_mask = syn_dists <= r_k                                     # (n_syn,) bool
        x_syn_valid = x_syn[valid_mask]                                   # (m, D)

        m = x_syn_valid.shape[0]
        if m > 0:
            syn_embs_list.append(x_syn_valid)
            syn_labels_list.extend([int(k)] * m)

    if len(syn_embs_list) == 0:
        return cp.empty((0, D), dtype=cp.float32), np.array([], dtype=np.int32)

    syn_embs_gpu = cp.vstack(syn_embs_list).astype(cp.float32)   # (M, D)
    syn_labels   = np.array(syn_labels_list, dtype=np.int32)      # (M,)

    return syn_embs_gpu, syn_labels


# ── Quick Sanity Check ─────────────────────────────────────────────────────
print("=" * 60)
print("🧪 SANITY CHECK: Uji fungsi CGA dengan satu cluster")
print("=" * 60)

test_k      = minority_ids[0] if len(minority_ids) > 0 else 0
test_size   = cluster_sizes[test_k]

syn_test, lbl_test = cga_gaussian_sampling_gpu(
    embeddings_gpu     = embeddings_gpu,
    labels             = labels_baseline,
    target_cluster_ids = np.array([test_k]),
    alpha              = 1.0,
    n_synthetic        = 5,
    seed               = Config.SEED
)

# Hitung jarak rata-rata sampel sintetis dari centroid cluster
orig_mask   = labels_baseline == test_k
mu_test     = embeddings_gpu[cp.array(orig_mask)].mean(axis=0)
syn_dists   = cp.linalg.norm(syn_test - mu_test[cp.newaxis, :], axis=1)
orig_dists  = cp.linalg.norm(
    embeddings_gpu[cp.array(orig_mask)] - mu_test[cp.newaxis, :], axis=1
)

print(f"""
  Cluster ID       : {test_k}
  Original members : {test_size}
  Synthetic diminta: 5
  Synthetic valid  : {len(syn_test)} (yang lolos boundary validation)

  Jarak ke centroid:
    Original (mean)  : {float(orig_dists.mean()):.4f}
    Synthetic (mean) : {float(syn_dists.mean()):.4f}
    → Synthetic berada {'dalam' if float(syn_dists.mean()) <= float(orig_dists.mean())*1.5 else 'luar'} jangkauan yang wajar

  Shape output GPU : {syn_test.shape}
  Dtype            : {syn_test.dtype}
""")
print("✅ Sanity check passed — fungsi CGA bekerja dengan benar")

## Cell 6 — Ablation Study: Grid Search α × n_synthetic

Menguji semua kombinasi hyperparameter CGA:
- **α (alpha)** ∈ {0.5, 1.0, 1.5}: seberapa besar variasi sampel sintetis
- **n_synthetic** ∈ {3, 5, 10}: berapa sampel yang dibuat per cluster

Untuk setiap kombinasi:
1. Jalankan CGA → hasilkan synthetic embeddings (GPU)
2. Gabungkan dengan embeddings asli
3. Re-run HDBSCAN dengan parameter sama
4. Evaluasi: Silhouette Score, jumlah cluster, coverage rate

In [ ]:
print("=" * 60)
print("🔬 ABLATION STUDY: α × n_synthetic")
print("=" * 60)
print(f"\n  Grid: α={Config.ALPHA_LIST} × n_syn={Config.N_SYNTHETIC_LIST}")
print(f"  Minority clusters target: {len(minority_ids)} clusters")
print(f"  Total kombinasi: {len(Config.ALPHA_LIST) * len(Config.N_SYNTHETIC_LIST)}")
print()

ablation_results = []

header = f"  {'α':>5} {'n_syn':>6} | {'n_syn_gen':>10} {'n_aug':>8} {'clusters':>9} {'noise':>7} {'coverage':>9} {'silhouette':>11}"
sep    = f"  {'─'*5} {'─'*6} | {'─'*10} {'─'*8} {'─'*9} {'─'*7} {'─'*9} {'─'*11}"
print(header)
print(sep)

for alpha, n_syn in product(Config.ALPHA_LIST, Config.N_SYNTHETIC_LIST):

    t_start = time.time()

    # ── Step 1: Generate synthetic embeddings di GPU ──────────────
    syn_embs_gpu, syn_labels = cga_gaussian_sampling_gpu(
        embeddings_gpu     = embeddings_gpu,
        labels             = labels_baseline,
        target_cluster_ids = minority_ids,
        alpha              = alpha,
        n_synthetic        = n_syn,
        seed               = Config.SEED
    )

    n_synthetic_generated = len(syn_embs_gpu)

    # ── Step 2: Gabungkan original + synthetic (GPU → CPU) ────────
    # Transfer synthetic ke CPU untuk HDBSCAN (correlation metric)
    syn_embs_cpu = cp.asnumpy(syn_embs_gpu)                          # (M, 512)
    emb_augmented = np.vstack([embeddings, syn_embs_cpu])             # (N+M, 512)
    n_total_aug = len(emb_augmented)

    # ── Step 3: Re-clustering HDBSCAN ─────────────────────────────
    clusterer_aug = hdbscan.HDBSCAN(
        min_cluster_size         = Config.RECLUSTER_PARAMS['min_cluster_size'],
        min_samples              = Config.RECLUSTER_PARAMS['min_samples'],
        cluster_selection_method = Config.RECLUSTER_PARAMS['method'],
        metric                   = Config.RECLUSTER_PARAMS['metric'],
    )
    labels_aug_full = clusterer_aug.fit_predict(emb_augmented)

    # Ambil hanya label untuk embeddings ASLI (bukan synthetic)
    labels_aug_orig = labels_aug_full[:len(embeddings)]

    # ── Step 4: Evaluasi ──────────────────────────────────────────
    n_clusters_aug = len(set(labels_aug_orig)) - 1
    n_noise_aug    = int((labels_aug_orig == -1).sum())
    coverage_aug   = float((labels_aug_orig != -1).sum() / len(labels_aug_orig))

    sil_aug = None
    mask_valid = labels_aug_orig != -1
    if mask_valid.sum() > 10 and len(set(labels_aug_orig[mask_valid])) >= 2:
        sil_aug = float(silhouette_score(
            embeddings[mask_valid], labels_aug_orig[mask_valid],
            metric='correlation', sample_size=3000, random_state=Config.SEED
        ))

    elapsed = time.time() - t_start

    ablation_results.append({
        'alpha'              : alpha,
        'n_synthetic'        : n_syn,
        'n_synthetic_gen'    : n_synthetic_generated,
        'n_total_aug'        : n_total_aug,
        'n_clusters'         : n_clusters_aug,
        'n_noise'            : n_noise_aug,
        'coverage'           : coverage_aug,
        'silhouette'         : sil_aug,
        'elapsed_s'          : elapsed,
        'labels_aug_orig'    : labels_aug_orig.copy(),
    })

    sil_str = f"{sil_aug:.4f}" if sil_aug else "   N/A"
    print(f"  {alpha:>5.1f} {n_syn:>6} | {n_synthetic_generated:>10,} {n_total_aug:>8,} "
          f"{n_clusters_aug:>9} {n_noise_aug:>7,} {coverage_aug:>9.1%} {sil_str:>11}")

    # Bersihkan memori GPU
    del syn_embs_gpu, syn_embs_cpu, emb_augmented
    cp.get_default_memory_pool().free_all_blocks()

print(sep)
print(f"  {'BASELINE':>5} {'  —':>6} | {'       —':>10} {'—':>8} "
      f"{n_clusters_base:>9} {n_noise_base:>7,} {coverage_base:>9.1%} {'  (ref)':>11}")

# Temukan konfigurasi terbaik
df_ablation = pd.DataFrame([{k: v for k, v in r.items() if k != 'labels_aug_orig'}
                             for r in ablation_results])
df_valid    = df_ablation.dropna(subset=['silhouette'])

# Best by coverage
best_cov_idx  = df_valid['coverage'].idxmax()
best_cov      = df_valid.loc[best_cov_idx]

# Best by silhouette
best_sil_idx  = df_valid['silhouette'].idxmax()
best_sil      = df_valid.loc[best_sil_idx]

print(f"""
  ════════════════════════════════════════════════════════════
  🏆 BEST BY COVERAGE:
     α={best_cov['alpha']}, n_syn={int(best_cov['n_synthetic'])}
     Coverage  : {best_cov['coverage']:.1%}  (baseline: {coverage_base:.1%}, Δ={best_cov['coverage']-coverage_base:+.1%})
     Clusters  : {int(best_cov['n_clusters'])}  (baseline: {n_clusters_base})
     Silhouette: {best_cov['silhouette']:.4f}

  🏆 BEST BY SILHOUETTE:
     α={best_sil['alpha']}, n_syn={int(best_sil['n_synthetic'])}
     Silhouette: {best_sil['silhouette']:.4f}  (baseline tidak ada referensi sil di CGA context)
     Coverage  : {best_sil['coverage']:.1%}
     Clusters  : {int(best_sil['n_clusters'])}
  ════════════════════════════════════════════════════════════
""")

## Cell 7 — Analisis Best Configuration

Memilih konfigurasi CGA terbaik berdasarkan tradeoff antara **coverage** dan **silhouette score**, lalu menganalisis detail perubahannya.

In [ ]:
print("=" * 60)
print("📊 ANALISIS BEST CGA CONFIGURATION")
print("=" * 60)

# Pilih best berdasarkan coverage tertinggi
best_result = ablation_results[best_cov_idx]
labels_best = best_result['labels_aug_orig']

best_alpha   = best_result['alpha']
best_n_syn   = best_result['n_synthetic']
best_cov_val = best_result['coverage']
best_sil_val = best_result['silhouette']
best_nclu    = best_result['n_clusters']
best_nnoise  = best_result['n_noise']

# ── Perbandingan detail ────────────────────────────────────────────
print(f"""
  Konfigurasi terpilih: α={best_alpha}, n_synthetic={best_n_syn}

  ┌──────────────────────────────────────────────────────────────┐
  │          BASELINE          │          CGA                   │
  ├──────────────────────────────────────────────────────────────┤
  │  Clusters  : {n_clusters_base:<6}             │  Clusters  : {best_nclu:<6}           │
  │  Noise     : {n_noise_base:<6,}             │  Noise     : {best_nnoise:<6,}           │
  │  Coverage  : {coverage_base:.1%}              │  Coverage  : {best_cov_val:.1%}            │
  │  Silhouette: (ref)               │  Silhouette: {best_sil_val:.4f}           │
  ├──────────────────────────────────────────────────────────────┤
  │  Δ Clusters  : {best_nclu - n_clusters_base:+d}                                    │
  │  Δ Noise     : {best_nnoise - n_noise_base:+,}                                   │
  │  Δ Coverage  : {best_cov_val - coverage_base:+.1%}                                  │
  └──────────────────────────────────────────────────────────────┘
""")

# ── Analisis cluster yang berubah ─────────────────────────────────
sizes_base = Counter({k: v for k, v in Counter(labels_baseline).items() if k != -1})
sizes_best = Counter({k: v for k, v in Counter(labels_best).items() if k != -1})

# Cluster yang bertahan dan ukurannya berubah
common_clusters = set(sizes_base.keys()) & set(sizes_best.keys())
grown_clusters  = [(k, sizes_base[k], sizes_best[k])
                   for k in common_clusters if sizes_best[k] > sizes_base[k]]
shrunk_clusters = [(k, sizes_base[k], sizes_best[k])
                   for k in common_clusters if sizes_best[k] < sizes_base[k]]
new_clusters    = set(sizes_best.keys()) - set(sizes_base.keys())
lost_clusters   = set(sizes_base.keys()) - set(sizes_best.keys())

print(f"  Cluster yang bertambah besar    : {len(grown_clusters)}")
print(f"  Cluster yang berkurang (noise)  : {len(shrunk_clusters)}")
print(f"  Cluster baru (dari noise)       : {len(new_clusters)}")
print(f"  Cluster yang hilang             : {len(lost_clusters)}")

if grown_clusters:
    grown_clusters.sort(key=lambda x: x[2] - x[1], reverse=True)
    print(f"\n  Top cluster yang paling banyak bertambah anggota (setelah CGA):")
    print(f"  {'Cluster ID':>10} {'Before':>8} {'After':>8} {'Delta':>8}")
    print(f"  {'─'*10} {'─'*8} {'─'*8} {'─'*8}")
    for k, before, after in grown_clusters[:10]:
        delta = after - before
        print(f"  {k:>10} {before:>8} {after:>8} {delta:>+8}")

## Cell 8 — Visualisasi Hasil CGA

Empat panel visualisasi:
1. **Heatmap ablation** — coverage tiap kombinasi α × n_synthetic
2. **Bar chart perbandingan** — baseline vs CGA terbaik (clusters, noise, coverage)
3. **Distribusi ukuran cluster** — sebelum dan sesudah CGA
4. **Scatter ablation** — tradeoff coverage vs silhouette untuk tiap konfigurasi

In [ ]:
fig = plt.figure(figsize=(16, 12))
gs  = gridspec.GridSpec(2, 2, figure=fig, hspace=0.4, wspace=0.35)

ax1 = fig.add_subplot(gs[0, 0])  # Heatmap coverage ablation
ax2 = fig.add_subplot(gs[0, 1])  # Bar chart baseline vs CGA
ax3 = fig.add_subplot(gs[1, 0])  # Distribusi ukuran cluster
ax4 = fig.add_subplot(gs[1, 1])  # Scatter coverage vs silhouette

# ── Panel 1: Heatmap coverage ablation ────────────────────────────
pivot_cov = df_valid.pivot(index='alpha', columns='n_synthetic', values='coverage') * 100
im = ax1.imshow(pivot_cov.values, cmap='YlOrRd', aspect='auto',
                vmin=pivot_cov.values.min() - 1, vmax=pivot_cov.values.max() + 1)
ax1.set_xticks(range(len(pivot_cov.columns)))
ax1.set_xticklabels([f'n={v}' for v in pivot_cov.columns])
ax1.set_yticks(range(len(pivot_cov.index)))
ax1.set_yticklabels([f'α={v}' for v in pivot_cov.index])
ax1.set_title('Coverage Rate (%) per Konfigurasi CGA', fontweight='bold')
ax1.set_xlabel('n_synthetic')
ax1.set_ylabel('alpha (α)')
for i in range(len(pivot_cov.index)):
    for j in range(len(pivot_cov.columns)):
        val = pivot_cov.values[i, j]
        ax1.text(j, i, f'{val:.1f}%', ha='center', va='center',
                 color='black', fontsize=10, fontweight='bold')
plt.colorbar(im, ax=ax1, shrink=0.8)

# ── Panel 2: Bar chart baseline vs CGA ────────────────────────────
metrics      = ['Coverage (%)', 'Clusters', 'Noise (%)']
baseline_vals = [coverage_base * 100, n_clusters_base, (1 - coverage_base) * 100]
cga_vals      = [best_cov_val * 100, best_nclu, (1 - best_cov_val) * 100]
x = np.arange(len(metrics))
w = 0.35
b1 = ax2.bar(x - w/2, baseline_vals, w, label='Baseline', color='steelblue', edgecolor='black')
b2 = ax2.bar(x + w/2, cga_vals,      w, label=f'CGA (α={best_alpha}, n={best_n_syn})',
             color='darkorange', edgecolor='black')
ax2.set_xticks(x)
ax2.set_xticklabels(metrics)
ax2.set_title('Baseline vs CGA Terbaik', fontweight='bold')
ax2.legend()
for bar in list(b1) + list(b2):
    h = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2, h + 0.5,
             f'{h:.1f}', ha='center', va='bottom', fontsize=9)

# ── Panel 3: Distribusi ukuran cluster ────────────────────────────
sizes_base_vals = [cluster_sizes[k] for k in cluster_sizes]
sizes_best_vals = [sizes_best[k] for k in sizes_best]
bins = np.linspace(0, max(max(sizes_base_vals), max(sizes_best_vals)) + 10, 30)
ax3.hist(sizes_base_vals, bins=bins, alpha=0.6, label='Baseline', color='steelblue', edgecolor='white')
ax3.hist(sizes_best_vals, bins=bins, alpha=0.6, label=f'CGA (α={best_alpha}, n={best_n_syn})',
         color='darkorange', edgecolor='white')
ax3.axvline(Config.MINORITY_THRESHOLD, color='red', linestyle='--', linewidth=1.5,
            label=f'Minority threshold ({Config.MINORITY_THRESHOLD})')
ax3.set_xlabel('Ukuran Cluster (jumlah anggota)')
ax3.set_ylabel('Frekuensi')
ax3.set_title('Distribusi Ukuran Cluster\nSebelum vs Sesudah CGA', fontweight='bold')
ax3.legend()
ax3.yaxis.set_major_locator(MaxNLocator(integer=True))

# ── Panel 4: Scatter coverage vs silhouette ────────────────────────
colors = plt.cm.viridis(np.linspace(0, 1, len(Config.ALPHA_LIST)))
alpha_color_map = dict(zip(Config.ALPHA_LIST, colors))

for _, row in df_valid.iterrows():
    c = alpha_color_map[row['alpha']]
    ax4.scatter(row['coverage'] * 100, row['silhouette'],
                color=c, s=120, edgecolors='black', zorder=3)
    ax4.annotate(f"n={int(row['n_synthetic'])}",
                 (row['coverage'] * 100, row['silhouette']),
                 textcoords='offset points', xytext=(5, 5), fontsize=8)

# Highlight best
ax4.scatter(best_cov_val * 100, best_sil_val, s=250, marker='*',
            color='red', zorder=5, label=f'Best (α={best_alpha}, n={best_n_syn})')

# Legend untuk alpha
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor=alpha_color_map[a],
                           markersize=10, label=f'α={a}') for a in Config.ALPHA_LIST]
legend_elements.append(Line2D([0], [0], marker='*', color='w', markerfacecolor='red',
                               markersize=14, label='Best Config'))
ax4.legend(handles=legend_elements, loc='lower right')
ax4.set_xlabel('Coverage Rate (%)')
ax4.set_ylabel('Silhouette Score')
ax4.set_title('Tradeoff: Coverage vs Silhouette\nper Konfigurasi CGA', fontweight='bold')

fig.suptitle('Notebook 12: CGA — Cluster-Guided Gaussian Augmentation\nAblation Study Results',
             fontsize=14, fontweight='bold')

plot_path = f"{Config.PLOTS_DIR}notebook12_cga_results.png"
plt.savefig(plot_path, dpi=150, bbox_inches='tight')
print(f"💾 Plot disimpan: {plot_path}")
plt.show()

## Cell 9 — Summary & Simpan Hasil

In [ ]:
print("=" * 70)
print("📋 NOTEBOOK 12 — SUMMARY CGA")
print("=" * 70)

delta_cov      = best_cov_val - coverage_base
delta_clusters = best_nclu - n_clusters_base
delta_noise    = best_nnoise - n_noise_base

print(f"""
┌──────────────────────────────────────────────────────────────────────┐
│              CLUSTER-GUIDED GAUSSIAN AUGMENTATION (CGA)              │
│              RINGKASAN EKSPERIMEN                                    │
├──────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  DATASET:                                                            │
│  • Total embeddings asli   : {len(embeddings):,}                          │
│  • Minority clusters target: {len(minority_ids)} clusters                     │
│  • Minority threshold      : ≤ {Config.MINORITY_THRESHOLD} anggota                    │
│                                                                      │
│  ABLATION STUDY:                                                     │
│  • α values  : {Config.ALPHA_LIST}                               │
│  • n_syn vals: {Config.N_SYNTHETIC_LIST}                                         │
│  • Total kombinasi: {len(Config.ALPHA_LIST) * len(Config.N_SYNTHETIC_LIST)}                                    │
│                                                                      │
│  KONFIGURASI TERBAIK (by coverage):                                  │
│  • α={best_alpha}, n_synthetic={best_n_syn}                                     │
│  • Synthetic embeddings gen: {best_result['n_synthetic_gen']:,}                     │
│                                                                      │
│  PERBANDINGAN HASIL:                                                 │
│  ┌───────────────┬──────────────┬─────────────┬──────────────────┐  │
│  │ Metric        │   Baseline   │     CGA     │      Delta       │  │
│  ├───────────────┼──────────────┼─────────────┼──────────────────┤  │
│  │ Clusters      │ {n_clusters_base:<12} │ {best_nclu:<11} │ {delta_clusters:+16} │  │
│  │ Noise         │ {n_noise_base:<12,} │ {best_nnoise:<11,} │ {delta_noise:+16,} │  │
│  │ Coverage      │ {coverage_base:<12.1%} │ {best_cov_val:<11.1%} │ {delta_cov:+16.1%} │  │
│  │ Silhouette    │ (ref)        │ {best_sil_val:<11.4f} │                  │  │
│  └───────────────┴──────────────┴─────────────┴──────────────────┘  │
└──────────────────────────────────────────────────────────────────────┘
""")

# ── Simpan semua hasil ─────────────────────────────────────────────
save_data = {
    'timestamp'            : datetime.now().isoformat(),
    'notebook'             : 'Notebook 12: CGA — Cluster-Guided Gaussian Augmentation',
    'n_total_embeddings'   : len(embeddings),
    'n_minority_clusters'  : len(minority_ids),
    'minority_ids'         : minority_ids,
    'minority_threshold'   : Config.MINORITY_THRESHOLD,
    'baseline_params'      : Config.BASELINE_PARAMS,
    'n_clusters_baseline'  : n_clusters_base,
    'n_noise_baseline'     : n_noise_base,
    'coverage_baseline'    : coverage_base,
    'ablation_results'     : [{
        'alpha'            : r['alpha'],
        'n_synthetic'      : r['n_synthetic'],
        'n_synthetic_gen'  : r['n_synthetic_gen'],
        'n_clusters'       : r['n_clusters'],
        'n_noise'          : r['n_noise'],
        'coverage'         : r['coverage'],
        'silhouette'       : r['silhouette'],
        'elapsed_s'        : r['elapsed_s'],
    } for r in ablation_results],
    'best_alpha'           : best_alpha,
    'best_n_synthetic'     : best_n_syn,
    'best_labels'          : labels_best,
    'best_coverage'        : best_cov_val,
    'best_silhouette'      : best_sil_val,
    'best_n_clusters'      : best_nclu,
    'best_n_noise'         : best_nnoise,
    'delta_coverage'       : delta_cov,
    'delta_clusters'       : delta_clusters,
    'delta_noise'          : delta_noise,
    'gpu_seed'             : Config.SEED,
}

pkl_path = f"{Config.RESULTS_DIR}notebook12_cga_results.pkl"
with open(pkl_path, 'wb') as f:
    pickle.dump(save_data, f)

print(f"💾 Hasil disimpan: {pkl_path}")
print()
print("=" * 70)
print("✅ Notebook 12 selesai. Hasil CGA siap untuk analisis lanjutan.")
print("=" * 70)

# Bersihkan GPU memory
del embeddings_gpu
cp.get_default_memory_pool().free_all_blocks()
print(f"\n🧹 GPU memory dibersihkan.")